In [1]:
import json
import numpy as np
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import ADASYN
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support

In [2]:
features_dir = '../features/'
data = pd.read_csv(features_dir + 'train_test.csv')
cat_features = eval(open(features_dir + 'cat_cols').read())
cat_features.extend(eval(open(features_dir + 'cat_cols_expand').read()))
cat_features = list(set(cat_features))
cat_features = [i for i in cat_features if i in data.columns]
text_features = None
if 'opscope' in data.columns:
    text_features = ['opscope']
for i in cat_features:
    data[i] = data[i].astype('str')
data = data.fillna(0)

In [3]:
pd.DataFrame(data[data.label.notna()].label.value_counts())

,label
0.0,13884
1.0,981


In [10]:
# a = pd.DataFrame(data.iloc[:,1:-1].dtypes)
# b = []
# b.extend(list(a[a[0]=='object'].index))
# b.extend(list(a[a[0]=='bool'].index))
# b.extend(list(cat_features))
# cat_index_features = [(i in b) for i in data.iloc[:,1:-1].dtypes.index]
# sm = SMOTENC(categorical_features=cat_index_features, n_jobs=8)
# X, y = sm.fit_resample(data.iloc[:,1:-1], data.iloc[:,-1])

In [67]:
def training(X_train, y_train, test):
    cat = CatBoostClassifier(
        iterations=500, 
        silent=True
        # task_type='GPU',
        # dictionaries=[
        #     'Word:min_token_occurrence=5',
        # ],
        # text_processing=[
        #     'NaiveBayes+Word|BoW:top_tokens_count=1000+Word|BM25+Word'
        # ]
        )
    cat.fit(X_train, y=y_train,
        cat_features=cat_features,
        text_features=text_features, 
        eval_set=(X_test, y_test)
    )
    return cat

In [68]:
# X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,1:-1], data.iloc[:,-1], test_size=0.2)
# cat = training(X_train, y_train, (X_test, y_test))
# y_pred = pd.Series(cat.predict_proba(X_test)[:,1]).apply(lambda x:1 if x > 0.5 else 0)
# print(classification_report(y_test, y_pred))

In [77]:
X = data.iloc[:,1:-1]
y = data.iloc[:,-1]
skf = StratifiedKFold(shuffle=True)
report = []
report_dict = []
cat = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    cat.append(training(X_train, y_train, (X_test, y_test)))
    y_pred = pd.Series(cat[-1].predict_proba(X_test)[:,1]).apply(lambda x:1 if x > 0.5 else 0)
    report.append(classification_report(y_test, y_pred))
    report_dict.append(classification_report(y_test, y_pred, output_dict=True))
    print(report[-1], '\n')

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      2777
         1.0       0.82      0.85      0.83       196

    accuracy                           0.98      2973
   macro avg       0.91      0.92      0.91      2973
weighted avg       0.98      0.98      0.98      2973
 

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      2777
         1.0       0.83      0.86      0.85       196

    accuracy                           0.98      2973
   macro avg       0.91      0.93      0.92      2973
weighted avg       0.98      0.98      0.98      2973
 

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      2777
         1.0       0.84      0.86      0.85       196

    accuracy                           0.98      2973
   macro avg       0.92      0.93      0.92      2973
weighted avg       0.98      0.98      0.98      2973
 

             

In [70]:
# cat = training(data.iloc[:,1:-1], data.iloc[:,-1], None)

In [79]:
features_dir = '../features/'
eval_set = pd.read_csv(features_dir + 'evaluate.csv')
for i in cat_features:
    eval_set[i] = eval_set[i].astype('str')
evaluate = pd.DataFrame(eval_set.id)
evaluate['score'] = pd.DataFrame(cat[3].predict_proba(eval_set.iloc[:,1:])[:,1])
evaluate.to_csv('../evaluate.csv', index=False)